<a href="https://colab.research.google.com/github/sgsnehil99/facenet/blob/master/compare(change).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
Model_path = 'content/drive/My Drive/DATASET_face/20180402-114759/20180402-114759.pb' 

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
tf.__version__

'2.2.0'

In [5]:
!pip install facenet

     |████████████████████████████████| 2.0MB 4.9MB/s 


In [0]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [10]:
#from __future__ import absolute_import
#from __future__ import division
#from __future__ import print_function

from scipy import misc
import tensorflow as tf
import numpy as np
import sys
import os
import copy
import argparse
import facenet

sys.path.insert(0,'/content/drive/My Drive/DATASET_face/facenet-master/src/align/')
import detect_face

def main():
      
    image_files = '/content/drive/My Drive/DATASET_face/IH conference photo/'  
    image_size = 160
    margin = 44
    gpu_memory_fraction = 1.0
    images = load_and_align_data(image_files, image_size, margin, gpu_memory_fraction)
    with tf.Graph().as_default():

        with tf.compat.v1.Session() as sess:
      
            # Load the model
            facenet.load_model(Model_path)
    
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")

            # Run forward pass to calculate embeddings
            feed_dict = { images_placeholder: images, phase_train_placeholder:False }
            emb = sess.run(embeddings, feed_dict=feed_dict)
            print(emb)
            
            
            nrof_images = len(image_files)

            print('Images:')
            for i in range(nrof_images):
                print('%1d: %s' % (i, image_files[i]))
            print('')
            
            # Print distance matrix
            print('Distance matrix')
            print('    ', end='')
            for i in range(nrof_images):
                print('    %1d     ' % i, end='')
            print('')
            for i in range(nrof_images):
                print('%1d  ' % i, end='')
                for j in range(nrof_images):
                    dist = np.sqrt(np.sum(np.square(np.subtract(emb[i,:], emb[j,:]))))
                    print('  %1.4f  ' % dist, end='')
                print('')
            
            
def load_and_align_data(image_paths, image_size, margin, gpu_memory_fraction):

    minsize = 20 # minimum size of face
    threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
    factor = 0.709 # scale factor
    
    print('Creating networks and loading parameters')
    with tf.Graph().as_default():
        gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
        sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            pnet, rnet, onet = detect_face.create_mtcnn(sess, None)
  
    tmp_image_paths=copy.copy(image_paths)
    img_list = []
    for image in tmp_image_paths:
        img = misc.imread(os.path.expanduser(image), mode='RGB')
        img_size = np.asarray(img.shape)[0:2]
        bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
        #print(bounding_boxes)
        if len(bounding_boxes) < 1:
          image_paths.remove(image)
          print("can't detect face, remove ", image)
          continue
        det = np.squeeze(bounding_boxes[0,0:4])
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin/2, 0)
        bb[1] = np.maximum(det[1]-margin/2, 0)
        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
        cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
        aligned = misc.imresize(cropped, (image_size, image_size), interp='bilinear')
        #print(aligned)
        prewhitened = facenet.prewhiten(aligned)
        img_list.append(prewhitened)
    #print(bounding_boxes)    
    #print(aligned)
    images = np.stack(img_list)
    return images
'''
def parse_arguments():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('model', type=str, 
        help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
    parser.add_argument('image_files', type=str, nargs='+', help='Images to compare')
    parser.add_argument('--image_size', type=int,
        help='Image size (height, width) in pixels.', default=160)
    parser.add_argument('--margin', type=int,
        help='Margin for the crop around the bounding box (height, width) in pixels.', default=44)
    parser.add_argument('--gpu_memory_fraction', type=float,
        help='Upper bound on the amount of GPU memory that will be used by the process.', default=1.0)
    return parser.parse_args(argv)
'''


"\ndef parse_arguments():\n    parser = argparse.ArgumentParser()\n    \n    parser.add_argument('model', type=str, \n        help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')\n    parser.add_argument('image_files', type=str, nargs='+', help='Images to compare')\n    parser.add_argument('--image_size', type=int,\n        help='Image size (height, width) in pixels.', default=160)\n    parser.add_argument('--margin', type=int,\n        help='Margin for the crop around the bounding box (height, width) in pixels.', default=44)\n    parser.add_argument('--gpu_memory_fraction', type=float,\n        help='Upper bound on the amount of GPU memory that will be used by the process.', default=1.0)\n    return parser.parse_args(argv)\n"

In [14]:
main()  

Creating networks and loading parameters


AttributeError: ignored

In [9]:
!pip install --upgrade smart_open

Requirement already up-to-date: smart_open in /usr/local/lib/python3.6/dist-packages (2.0.0)
